# RAG
(retrieval-augmented generation)

In [12]:
from operator import itemgetter

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [13]:
text_input = "Liam Everest, born in Crestwood, showed early curiosity in tech and nature. \
        A Mechatronics graduate from Crestwood University, he excelled in eco-friendly tech. \
        Liam aspires to blend technology and nature for a harmonious future."

In [17]:
import chromadb

ModuleNotFoundError: No module named 'chromadb'

In [15]:
persist_directory = './Data/chroma/'

In [16]:
vectorstore = Chroma.from_texts(
    [text_input], embedding=OpenAIEmbeddings(), persist_directory=persist_directory
)
retriever = vectorstore.as_retriever()

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.